# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [1]:
from pyspark.sql import SparkSession


In [2]:
spark=SparkSession.builder.appName('dogfood').getOrCreate()

In [3]:
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier, GBTClassifier

In [4]:
input_data=spark.read.csv('dog_food.csv',header=True, inferSchema=True)

In [5]:
input_data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [6]:
input_data.head()

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)

In [7]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler=VectorAssembler(inputCols=['A','B','C','D'],outputCol='features')

In [11]:
input_data=assembler.transform(input_data)

In [12]:
train,test=input_data.randomSplit([0.7,0.3])

In [13]:
train.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [29]:
model_dt=DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')
model_rf=RandomForestClassifier(labelCol='Spoiled',featuresCol='features')

In [30]:
train=train.select('features','Spoiled')
model_dt=model_dt.fit(train)
model_rf=model_rf.fit(train)

In [31]:
dt_preds=model_dt.transform(test)
rf_preds=model_dt.transform(test)

In [32]:
dt_preds.show()
rf_preds.show()

+---+---+----+---+-------+-------------------+-------------+-----------+----------+
|  A|  B|   C|  D|Spoiled|           features|rawPrediction|probability|prediction|
+---+---+----+---+-------+-------------------+-------------+-----------+----------+
|  1|  1|12.0|  4|    1.0| [1.0,1.0,12.0,4.0]|   [0.0,68.0]|  [0.0,1.0]|       1.0|
|  1|  3| 8.0|  5|    0.0|  [1.0,3.0,8.0,5.0]|  [194.0,0.0]|  [1.0,0.0]|       0.0|
|  1|  5|13.0| 10|    1.0|[1.0,5.0,13.0,10.0]|   [0.0,68.0]|  [0.0,1.0]|       1.0|
|  1|  6|11.0| 10|    1.0|[1.0,6.0,11.0,10.0]|    [0.0,2.0]|  [0.0,1.0]|       1.0|
|  1|  7| 7.0|  2|    0.0|  [1.0,7.0,7.0,2.0]|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  1|  7| 8.0|  2|    0.0|  [1.0,7.0,8.0,2.0]|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  1|  8| 7.0| 10|    0.0| [1.0,8.0,7.0,10.0]|  [194.0,0.0]|  [1.0,0.0]|       0.0|
|  1|  9| 7.0|  4|    0.0|  [1.0,9.0,7.0,4.0]|  [194.0,0.0]|  [1.0,0.0]|       0.0|
|  1| 10|11.0|  4|    1.0|[1.0,10.0,11.0,4.0]|    [0.0,2.0]|  [0.0,1.0]|    

In [33]:
model_dt.featureImportances

SparseVector(4, {0: 0.0188, 1: 0.0169, 2: 0.9433, 3: 0.0209})

In [34]:
model_rf.featureImportances

SparseVector(4, {0: 0.0341, 1: 0.0273, 2: 0.9119, 3: 0.0268})

In [35]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [36]:
evaluation=BinaryClassificationEvaluator(labelCol='Spoiled')

In [37]:
evaluation.evaluate(dt_preds)

0.9760846560846561

In [39]:
evaluation.evaluate(rf_preds)

0.9760846560846561